## Default Rate Estimation using LightGBM on Spark

### Introduction
As we known, `LightGBM` is a very popular machine learning library in the data competitions and industries because of its excellent effect and interpretability. In this notebook, we will use `Synapse LightGBM` to build our binary classification model for dataset of [Tianchi Competetion](https://tianchi.aliyun.com/competition/entrance/531830/information), which can run on Spark and utilize cluster computing power to train, evaluate and tune the model.

### Initialize Spark and Read Dataset

In this section, we need init our Spark session and read training dataset stored in `${MY_S3_BUCKET}/risk/tianchi/fg_train_data.csv`. Moreover, it may consume a little more time due to the need to download the `Synapse LightGBM`. You may need a http/https proxy server to speed up the download process as below:

```python
spark = pyspark.sql.SparkSession.builder\
    .appName("Loan Default Estimation-LightGBM") \
    ...
    .config("spark.driver.extraJavaOptions", "-Dhttp.proxyHost=<proxyHost> -Dhttp.proxyPort=<proxyPort> -Dhttps.proxyHost=<proxyHost> -Dhttps.proxyPort=<proxyPort>") \
    .config("spark.jars.packages", "com.microsoft.azure:synapseml_2.12:0.9.4") \
    .config("spark.jars.repositories", "https://mmlspark.azureedge.net/maven") \
    ...
```

In [1]:
import pyspark
import yaml
import argparse
import onnxmltools
import subprocess
import lightgbm as lgb
import numpy as np
import pandas as pd
import warnings

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import VectorAssembler

warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
def init_spark():
    spark = pyspark.sql.SparkSession.builder\
            .appName("Loan Default Estimation-LightGBM") \
            .config("spark.executor.memory","8G") \
            .config("spark.executor.instances","4") \
            .config("spark.executor.cores", "4") \
            .config("spark.jars.packages", "com.microsoft.azure:synapseml_2.12:0.9.4") \
            .config("spark.jars.repositories", "https://mmlspark.azureedge.net/maven") \
            .getOrCreate()
    sc = spark.sparkContext
    print(sc.version)
    print(sc.applicationId)
    print(sc.uiWebUrl)
    return spark

def load_config(path):
    params = dict()
    with open(path, 'r') as stream:
        params = yaml.load(stream, Loader=yaml.FullLoader)
    return params

def read_dataset(spark, data_path):
    dataset = spark.read.format("csv")\
      .option("header",  True)\
      .option("inferSchema",  True)\
      .load(data_path)  
    return dataset

def get_vectorassembler(dataset, features='features', label='label'):
    featurizer = VectorAssembler(
        inputCols = feature_cols,
        outputCol = 'features',
        handleInvalid = 'skip'
    )
    dataset = featurizer.transform(dataset)[label, features]
    return dataset

In [3]:
params = load_config('../conf/spark_lgbm_dev.yaml')

In [4]:
spark = init_spark()

https://mmlspark.azureedge.net/maven added as a remote repository with the name: repo-1
Ivy Default Cache set to: /home/spark/.ivy2/cache
The jars for the packages stored in: /home/spark/.ivy2/jars
com.microsoft.azure#synapseml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5364f257-84ed-40f5-b9e6-8101e06cb45e;1.0
	confs: [default]


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found com.microsoft.azure#synapseml_2.12;0.9.4 in repo-1
	found com.microsoft.azure#synapseml-core_2.12;0.9.4 in repo-1
	found org.scalactic#scalactic_2.12;3.0.5 in central
	found org.scala-lang#scala-reflect;2.12.4 in central
	found io.spray#spray-json_2.12;1.3.2 in central
	found com.jcraft#jsch;0.1.54 in central
	found org.apache.httpcomponents#httpclient;4.5.6 in central
	found org.apache.httpcomponents#httpcore;4.4.10 in central
	found commons-logging#commons-logging;1.2 in central
	found commons-codec#commons-codec;1.10 in central
	found org.apache.httpcomponents#httpmime;4.5.6 in central
	found com.linkedin.isolation-forest#isolation-forest_3.0.0_2.12;1.0.1 in central
	found com.chuusai#shapeless_2.12;2.3.2 in central
	found org.typelevel#macro-compat_2.12;1.1.1 in central
	found org.apache.spark#spark-avro_2.12;3.0.0 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.testng#testng;6.8.8 in central
	found org.beanshell#bsh;2.0b4 in central
	found com.b

3.1.2
spark-application-1653998207409
http://jupyter.my.nginx.test/hub/user-redirect/proxy/4040/jobs/


In [5]:
data_path = params['fg_train_dataset_path']
fg_train_dataset = read_dataset(spark, data_path)

In [6]:
fg_train_dataset.limit(10).toPandas()

22/05/31 11:57:03 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,loanAmnt,term,interestRate,installment,grade,subGrade,employmentTitle,employmentLength,homeOwnership,annualIncome,verificationStatus,isDefault,purpose,postCode,regionCode,dti,delinquency_2years,ficoRangeLow,ficoRangeHigh,openAcc,pubRec,pubRecBankruptcies,revolBal,revolUtil,totalAcc,initialListStatus,applicationType,earliesCreditLine,title,policyCode,n0,n1,n2,n3,n4,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14,issueDateDT,grade_target_mean,subGrade_target_mean,grade_to_mean_n0,grade_to_std_n0,grade_to_mean_n1,grade_to_std_n1,grade_to_mean_n2,grade_to_std_n2,grade_to_mean_n4,grade_to_std_n4,grade_to_mean_n5,grade_to_std_n5,grade_to_mean_n6,grade_to_std_n6,grade_to_mean_n7,grade_to_std_n7,grade_to_mean_n8,grade_to_std_n8,grade_to_mean_n9,grade_to_std_n9,grade_to_mean_n10,grade_to_std_n10,grade_to_mean_n11,grade_to_std_n11,grade_to_mean_n12,grade_to_std_n12,grade_to_mean_n13,grade_to_std_n13,grade_to_mean_n14,grade_to_std_n14
0,35000.0,5,19.52,917.97,5,21,161280,2,2,110000.0,2,1,1,43,32,17.05,0.0,730.0,734.0,7.0,0.0,0.0,24178.0,48.9,27.0,0,0,2001,1,1.0,0.0,2.0,2.0,2.0,4.0,9.0,8.0,4.0,12.0,2.0,7.0,0.0,0.0,0.0,2.0,2587,0.386234,0.380444,1.876011,3.992386,1.874620,4.053876,1.942294,4.023418,1.869160,3.948124,1.897562,4.055665,1.865760,4.017884,1.840872,4.074681,1.851544,4.040923,1.938318,4.024912,1.842210,4.108917,1.852810,4.009823,1.852810,4.009823,1.857394,4.005352,1.856379,3.991791
1,18000.0,5,18.49,461.90,4,16,89538,5,0,46000.0,2,0,0,64,18,27.83,0.0,700.0,704.0,13.0,0.0,0.0,15096.0,38.9,18.0,1,0,2002,5768,1.0,0.0,3.0,5.0,5.0,10.0,7.0,7.0,7.0,13.0,5.0,13.0,0.0,0.0,0.0,2.0,1888,0.304227,0.298190,1.500809,3.193909,1.502905,3.185919,1.504054,3.173189,1.567352,3.204484,1.511316,3.139166,1.515599,3.098975,1.500817,3.139721,1.517874,3.086106,1.504140,3.174194,1.484104,3.173687,1.482248,3.207858,1.482248,3.207858,1.485915,3.204282,1.485103,3.193433
2,12000.0,5,16.99,298.17,4,17,159367,8,0,74000.0,2,0,0,265,14,22.77,0.0,675.0,679.0,11.0,0.0,0.0,4606.0,51.8,27.0,0,0,2006,0,1.0,0.0,0.0,3.0,3.0,0.0,0.0,21.0,4.0,5.0,3.0,11.0,0.0,0.0,0.0,4.0,3044,0.304227,0.302541,1.500809,3.193909,1.360761,2.998190,1.532981,3.241462,1.273891,3.071276,1.162371,3.176718,1.480241,3.125317,1.472698,3.259745,1.406712,3.254085,1.530998,3.244609,1.504230,3.089208,1.482248,3.207858,1.482248,3.207858,1.485915,3.204282,1.315111,3.146801
3,2050.0,3,7.69,63.95,1,3,59830,9,0,35000.0,0,0,0,465,14,17.49,0.0,755.0,759.0,12.0,0.0,0.0,3111.0,8.5,23.0,0,0,2006,0,1.0,0.0,1.0,3.0,3.0,7.0,11.0,3.0,10.0,18.0,3.0,12.0,0.0,0.0,0.0,3.0,2679,0.059838,0.065532,0.375202,0.798477,0.368239,0.796491,0.383245,0.810366,0.380622,0.806605,0.384972,0.802575,0.368526,0.819126,0.369865,0.798404,0.377964,0.799464,0.382750,0.811152,0.370128,0.799459,0.370562,0.801965,0.370562,0.801965,0.371479,0.801070,0.344287,0.793451
4,11500.0,3,14.98,398.54,3,12,85242,1,1,30000.0,2,0,0,3,4,32.60,0.0,665.0,669.0,8.0,1.0,1.0,14021.0,59.7,33.0,1,0,1994,0,1.0,0.0,4.0,4.0,4.0,4.0,16.0,10.0,5.0,21.0,4.0,8.0,0.0,0.0,0.0,2.0,2406,0.224522,0.224686,1.125607,2.395431,1.113406,2.430896,1.133984,2.439745,1.121496,2.368874,1.197930,2.401168,1.120956,2.388727,1.106851,2.450979,1.144817,2.403154,1.133458,2.441340,1.104961,2.446307,1.111686,2.405894,1.111686,2.405894,1.114436,2.403211,1.113827,2.395075
5,12000.0,3,12.99,404.27,3,11,65718,5,2,60000.0,1,1,0,770,13,19.22,0.0,690.0,694.0,15.0,0.0,0.0,27176.0,46.0,21.0,1,0,1994,0,1.0,0.0,7.0,13.0,13.0,7.0,7.0,2.0,13.0,17.0,11.0,15.0,0.0,0.0,0.0,6.0,3257,0.224522,0.204005,1.125607,2.395431,1.085997,2.408741,0.984707,2.361605,1.141867,2.419815,1.133487,2.354374,1.100101,2.459716,1.119411,2.396658,1.136053,2.409156,1.011351,2.376224,1.124941,2.384061,1.111686,2.405894,1.111686,2.405894,1.114436,2.403211,0.923430,2.361914
6,24000.0,3,9.99,774.30,2,7,209276,10,0,150000.0,1,0,2,40,8,5.68,0.0,690.0,694.0,7.0,0.0,0.0,4334.0,68.8,25.0,0,0,1983,18780,1.0,1.0,1.0,3.0,3.0,2.0,7.0,7.0,6.0,17.0,3.0,7.0,0.0,0.0,0.0,2.0,2983,0.131210,0.128111,0.707941,1.635584,0.736477,1.592982,0.766491,1.620731,0.720818,1.6

### Label and Features 
Suppose the Spark Dataframe of this training dataset only contains numerical features. Here we use `params['label']` column value as label and other columns as features.

In [7]:
feature_cols = [x for x in fg_train_dataset.columns if x not in [params['label']]]

In [8]:
feature_cols

['loanAmnt',
 'term',
 'interestRate',
 'installment',
 'grade',
 'subGrade',
 'employmentTitle',
 'employmentLength',
 'homeOwnership',
 'annualIncome',
 'verificationStatus',
 'purpose',
 'postCode',
 'regionCode',
 'dti',
 'delinquency_2years',
 'ficoRangeLow',
 'ficoRangeHigh',
 'openAcc',
 'pubRec',
 'pubRecBankruptcies',
 'revolBal',
 'revolUtil',
 'totalAcc',
 'initialListStatus',
 'applicationType',
 'earliesCreditLine',
 'title',
 'policyCode',
 'n0',
 'n1',
 'n2',
 'n3',
 'n4',
 'n5',
 'n6',
 'n7',
 'n8',
 'n9',
 'n10',
 'n11',
 'n12',
 'n13',
 'n14',
 'issueDateDT',
 'grade_target_mean',
 'subGrade_target_mean',
 'grade_to_mean_n0',
 'grade_to_std_n0',
 'grade_to_mean_n1',
 'grade_to_std_n1',
 'grade_to_mean_n2',
 'grade_to_std_n2',
 'grade_to_mean_n4',
 'grade_to_std_n4',
 'grade_to_mean_n5',
 'grade_to_std_n5',
 'grade_to_mean_n6',
 'grade_to_std_n6',
 'grade_to_mean_n7',
 'grade_to_std_n7',
 'grade_to_mean_n8',
 'grade_to_std_n8',
 'grade_to_mean_n9',
 'grade_to_std_n9',


In [9]:
train_data = get_vectorassembler(fg_train_dataset, label=params['label'], features='features')

In [10]:
train_data.limit(10).toPandas()

,isDefault,features
0,1,"[35000.0, 5.0, 19.52, 917.97, 5.0, 21.0, 16128..."
1,0,"[18000.0, 5.0, 18.49, 461.9, 4.0, 16.0, 89538...."
2,0,"[12000.0, 5.0, 16.99, 298.17, 4.0, 17.0, 15936..."
3,0,"[2050.0, 3.0, 7.69, 63.95, 1.0, 3.0, 59830.0, ..."
4,0,"[11500.0, 3.0, 14.98, 398.54, 3.0, 12.0, 85242..."
5,1,"[12000.0, 3.0, 12.99, 404.27, 3.0, 11.0, 65718..."
6,0,"[24000.0, 3.0, 9.99, 774.3, 2.0, 7.0, 209276.0..."
7,0,"[16000.0, 3.0, 7.91, 500.72, 1.0, 4.0, 8198.0,..."
8,0,"[6000.0, 3.0, 10.49, 194.99, 2.0, 6.0, 115263...."
9,0,"[10375.0, 5.0, 15.61, 250.16, 4.0, 15.0, 74728..."


In [11]:
train_data.count()

612742

In [12]:
train_data[train_data[params['label']]==1].count()

119541

In [13]:
train, valid = train_data.randomSplit([0.80, 0.20], seed=1)

### Train and evaluation
In this section, we will use Synapse LightGBM to build our binary classification model. The meaning of model hyper parameters can be referred to:
* https://mmlspark.blob.core.windows.net/docs/0.9.5/pyspark/synapse.ml.lightgbm.html#module-synapse.ml.lightgbm.LightGBMClassifier
* https://lightgbm.readthedocs.io/en/latest/Parameters.html


In [14]:
hyper_params =  {
    'boostingType':'gbdt',
    'objective':'binary',
    'metric':'auc',
    'numLeaves': 2**5,
    'lambdaL1':10,
    'lambdaL2':10,
    'maxDepth':-1,
    'minDataInLeaf':20,
    'minSumHessianInLeaf':0.001,
    'minGainToSplit':0.0,
    'featureFraction':0.8,
    'baggingFraction':0.8,
    'baggingFreq':4,
    'learningRate':0.1,
    'numIterations':500,
    'earlyStoppingRound':100,
    'verbosity':1,
    'numThreads':16,
}

In [15]:
from synapse.ml.lightgbm import LightGBMClassifier
model = LightGBMClassifier(isProvideTrainingMetric=True, featuresCol="features", labelCol="isDefault", isUnbalance=True, **hyper_params)
model = model.fit(train)

In [16]:
print("train dataset prediciton")
predictions = model.transform(train)
predictions.show(3, False)

from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = pyspark.ml.evaluation.BinaryClassificationEvaluator(labelCol="isDefault",metricName="areaUnderROC")
auc = evaluator.evaluate(predictions)
print("train dataset auc:", auc)

train dataset prediciton


22/05/31 11:58:05 WARN DAGScheduler: Broadcasting large task binary with size 1844.3 KiB
22/05/31 11:58:06 WARN DAGScheduler: Broadcasting large task binary with size 1857.0 KiB


+---------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------+----------------------------------------+----------+
|isDefault|features                                                                       

train dataset auc: 0.7706209993721579


In [17]:
print("validation dataset prediciton")
predictions = model.transform(valid)
predictions.show(3, False)

from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = pyspark.ml.evaluation.BinaryClassificationEvaluator(labelCol="isDefault",metricName="areaUnderROC")
auc = evaluator.evaluate(predictions)
print("validation dataset auc:", auc)

validation dataset prediciton


22/05/31 11:58:13 WARN DAGScheduler: Broadcasting large task binary with size 1844.3 KiB
22/05/31 11:58:15 WARN DAGScheduler: Broadcasting large task binary with size 1857.0 KiB


+---------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------+----------------------------------------+----------+
|isDefault|features                                                                          

validation dataset auc: 0.7321580860915377


### Feature Importance

In [18]:
importance_df = (
    pd.DataFrame({
        'feature_name': feature_cols,
        'importance_gain': model.getFeatureImportances('gain'),
        'importance_split': model.getFeatureImportances('split'),
    })
    .sort_values('importance_gain', ascending=False)
    .reset_index(drop=True)
)
print(importance_df)

            feature_name  importance_gain  importance_split
0               subGrade     83997.066126             144.0
1        grade_to_std_n4     81705.719417             140.0
2       grade_to_mean_n4     76956.117708             112.0
3   subGrade_target_mean     57321.622027              32.0
4            issueDateDT     57033.936851             941.0
5                    dti     29891.142450             798.0
6           annualIncome     29267.896991             780.0
7                   term     28744.529145             124.0
8               loanAmnt     28052.935027             921.0
9       grade_to_mean_n7     27357.709897             129.0
10              revolBal     21181.492528             741.0
11           installment     20755.766657             649.0
12       employmentTitle     20452.801992             713.0
13         homeOwnership     19117.934570             209.0
14          ficoRangeLow     17453.734251             408.0
15            regionCode     17303.52335

### Hyper Parameters Tuning
In this section, we will use `Hyperopt` to tune the hyper parameters of the lightgbm model. Since the hyper parameter combination space is very large, we only demonstrate the search process of the `learningRate` and `numIterations`. If there is enough computing resources, more larger space and iteration rounds can be utilized to get better performance.

Here we use `hyperopt.tpe.suggest`, a Bayesian approach to search in the parameter combination space. For more information, please refer to the Jupyter Notebook of Databriks and the documentation of Hyperopt:
 * https://docs.databricks.com/_static/notebooks/hyperopt-spark-ml.html
 * http://hyperopt.github.io/hyperopt/#algorithms

In [19]:
import numpy as np
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# hyper paramaters template
hyper_params =  {
    'boostingType':'gbdt',
    'objective':'binary',
    'metric':'auc',
    'numLeaves': 2**5,
    'lambdaL1':10,
    'lambdaL2':10,
    'maxDepth':-1,
    'minDataInLeaf':20,
    'minSumHessianInLeaf':0.001,
    'minGainToSplit':0.0,
    'featureFraction':0.8,
    'baggingFraction':0.8,
    'baggingFreq':4,
    'learningRate':0.1,
    'numIterations':500,
    'earlyStoppingRound':100,
    'verbosity':1,
    'numThreads':16,
}

# define a function to minimize
def train_with_hyperopt(params, hyper_params, train, valid):
    """
    An example train method that calls into MLlib.
    This method is passed to hyperopt.fmin().

    :param params: hyperparameters as a dict. Its structure is consistent with how search space is defined. See below.
    :return: dict with fields 'loss' (scalar loss) and 'status' (success/failure status of run)
    """
    # For integer parameters, make sure to convert them to int type if Hyperopt is searching over a continuous range of values.
    hyper_params['learningRate'] = params['learningRate']
    hyper_params['numIterations'] = int(params['numIterations'])
    # train lightgbm model
    model = LightGBMClassifier(isProvideTrainingMetric=True, featuresCol="features", labelCol="isDefault", isUnbalance=True, **hyper_params)
    model = model.fit(train)
    # transform validation dataset
    predictions = model.transform(valid)
    evaluator = BinaryClassificationEvaluator(labelCol="isDefault",metricName="areaUnderROC")
    # evaluate auc
    auc = evaluator.evaluate(predictions)
    # Hyperopt expects you to return a loss (for which lower is better), so take the negative of the f1_score (for which higher is better).
    return {'loss': -auc, 'status': STATUS_OK}

# define the search space over hyperparameters
space = {
  'learningRate': hp.uniform('learningRate', 0.01, 0.1),
  'numIterations': hp.uniform('numIterations', 500, 2000),
}

In [20]:
from functools import partial
best_params = fmin(
    fn=partial(train_with_hyperopt, hyper_params=hyper_params, train=train, valid=valid),
    space=space,
    algo=tpe.suggest,
    max_evals=5
)

  0%|          | 0/5 [00:00<?, ?trial/s, best loss=?]

22/05/31 12:01:32 WARN DAGScheduler: Broadcasting large task binary with size 6.5 MiB


 20%|██        | 1/5 [03:18<13:12, 198.06s/trial, best loss: -0.7316463645843494]

22/05/31 12:04:11 WARN DAGScheduler: Broadcasting large task binary with size 5.4 MiB


 40%|████      | 2/5 [05:56<08:44, 174.93s/trial, best loss: -0.732994601578312] 

...

 60%|██████    | 3/5 [07:04<04:11, 125.92s/trial, best loss: -0.732994601578312]

22/05/31 12:07:33 WARN DAGScheduler: Broadcasting large task binary with size 4.5 MiB


 80%|████████  | 4/5 [09:17<02:08, 128.93s/trial, best loss: -0.734095579152508]

22/05/31 12:10:14 WARN DAGScheduler: Broadcasting large task binary with size 5.6 MiB


100%|██████████| 5/5 [12:00<00:00, 144.02s/trial, best loss: -0.734095579152508]


In [21]:
print(best_params)

{'learningRate': 0.023764779523899424, 'numIterations': 1273.8549043018331}


### Retrain the model on the full training dataset
In this section, we should use the full training dataset and evaluate the model effect on the test dataset. However, since we do not have labeled test dateset, we can only use the same traing data as before.

In [24]:
hyper_params =  {
    'boostingType':'gbdt',
    'objective':'binary',
    'metric':'auc',
    'numLeaves': 2**5,
    'lambdaL1':10,
    'lambdaL2':10,
    'maxDepth':-1,
    'minDataInLeaf':20,
    'minSumHessianInLeaf':0.001,
    'minGainToSplit':0.0,
    'featureFraction':0.8,
    'baggingFraction':0.8,
    'baggingFreq':4,
    'learningRate':best_params['learningRate'],
    'numIterations':int(best_params['numIterations']),
    'earlyStoppingRound':100,
    'verbosity':1,
    'numThreads':16,
}

model = LightGBMClassifier(isProvideTrainingMetric=True, featuresCol="features", labelCol="isDefault", isUnbalance=True, **hyper_params)
model = model.fit(train)

predictions = model.transform(train)
evaluator = pyspark.ml.evaluation.BinaryClassificationEvaluator(labelCol="isDefault",metricName="areaUnderROC")
auc = evaluator.evaluate(predictions)
print("train dataset auc:", auc)

predictions = model.transform(valid)
evaluator = pyspark.ml.evaluation.BinaryClassificationEvaluator(labelCol="isDefault",metricName="areaUnderROC")
auc = evaluator.evaluate(predictions)
print("validation dataset auc:", auc)

22/05/31 12:19:56 WARN DAGScheduler: Broadcasting large task binary with size 4.5 MiB


train dataset auc: 0.7579260854903618


22/05/31 12:20:04 WARN DAGScheduler: Broadcasting large task binary with size 4.5 MiB


validation dataset auc: 0.7339783176165244


In [25]:
spark.stop()

### Acknowledgement
Thanks to the Tianchi community for providing the loan default dataset and corresponding tutorial for risk management based on this dataset.